# Metody oparte o sąsiedztwo

## Konfiguracja

Laboratorium składa się z 2 części:
1. kNN w uczeniu nadzorowanym
2. Wyszukiwanie

W pierwszej części użyjemy standardowych bibliotek oraz dodatkowo:
- `pynndescent` dla approximate nearest neighbors (ANN)
- `pyarrow` do odczytu zbioru danych w formacie Parquet

W drugiej części natomiast do wektoryzacji obiektów użyjemy stosu do uczenia głębokiego opartego o PyTorcha i torchvision. Dodatkowo użyjemy:
- `sentence-transformers` - podobna do popularnej biblioteki Transformers, ale obsługuje głównie modele do uczenia nienadzorowanego, niewymagające fine-tuningu
- `tqdm` dla wyświetlania progress bar

**Uwaga:** trzeba użyć wersji Numpy'a 1.23.X, bo wersja 1.24 wprowadziła dużo breaking changes i inne biblioteki (np. PyNNDescent) jeszcze nie są zaktualizowane.

PyTorcha i torchvision zainstaluj [w wersji odpowiedniej dla twojego systemu i środowiska](https://pytorch.org/get-started/locally/). GPU nie będzie nam potrzebne, ale możesz go użyć, jeżeli jest odpowiednio skonfigurowane.

In [ ]:
# !pip install numpy==1.23.5 pandas matplotlib scikit-learn missingno pyarrow pynndescent sentence-transformers tqdm

In [ ]:
# !pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Zbiór danych - klasyfikacja danych numerycznych

Wykorzystamy zbiór danych [Codon usage](https://archive.ics.uci.edu/ml/datasets/Codon+usage) z dziedziny bioinformatyki. Został on zaprezentowany w artykule ([link do wersji Open Access](https://www.nature.com/articles/s41598-023-28965-7):

Hallee, Logan, and Bohdan B. Khomtchouk. *"Machine learning classifiers predict key genomic and evolutionary traits across the kingdoms of life."* Scientific Reports 13.1 (2023): 2088.

Autorzy wykonywali na nim wiele analiz, ale nas interesuje podstawowa - przewidywanie, z jakiego królestwa (kingdom) pochodzi komórka, na podstawie rozkładu kodonów w jej sekwencjonowanym RNA. Analiza DNA oraz RNA jest podstawowym zadaniem bioinformatyki. Kodony (codons) to trójki nukleotydów, np. UGC, i większość zapisuje pewien aminokwas (z wyjątkiem trzech kodonów stopu), który jest wykorzystywany w syntezie białek.

Problem jest taki, że w praktyce trzeba sekwencjonować DNA/RNA z wielu komórek, więc trzeba je namnożyć w laboratorium. Niestety nie da się uzyskać idealnej czystości, i zawsze jest szansa, że próbka zostanie zanieczyszczona, np. bakteriami. Wtedy oprócz interesujących nas komórek (np. zwierzęcych) otrzymamy inne, których trzeba się pozbyć przed dalszą analizą. Tutaj właśnie wchodzi ML - dokonamy klasyfikacji, z jakiego królestwa (a właściwie domeny) pochodzi kod RNA, żeby pomóc w takich sytuacjach.

Dla uproszczenia, autorzy, zamiast klasyfikować królestwo, klasyfikują [domenę](https://pl.wikipedia.org/wiki/Domena_(biologia)) organizmu. Są to jednostki o poziom wyżej w systematyce taksonomicznej. Do tego dokładamy wirusy i bakteriofagi. Mamy zatem klasy:
1. Archaea ([archeony](https://pl.wikipedia.org/wiki/Archeony))
2. Bacteria ([bakterie](https://pl.wikipedia.org/wiki/Bakterie))
3. Eukariota ([eukarionty](https://pl.wikipedia.org/wiki/Eukarionty)) - między innymi ludzie
4. Viruses ([wirusy](https://pl.wikipedia.org/wiki/Wirusy)) - nie są organizmami żywymi, więc nie są częścią typowej taksonomii, ale są ważne w badaniach mikrobiologicznych
5. Bacteriophages ([bakteriofagi](https://pl.wikipedia.org/wiki/Bakteriofag)) - rodzaj wirusów atakujących tylko bakterie, używane m.in. [w badaniach nad mikrobiomem jelitowym](https://kids.frontiersin.org/articles/10.3389/frym.2019.00146)

Szczegółowy opis zbioru znajduje się [na stronie UCI](https://archive.ics.uci.edu/ml/datasets/Codon+usage). W skrócie:
1. `Kingdom` - królestwo, z którego pochodzi komórka. Podział jest tutaj dość szczegółowy, ale organizmy te można pogrupować w praktyce na bakterie (bacteria), wirusy (viruses) i eukarionty (eukariota).
2. `DNAtype` - używana w innych analizach w artykule.
3. `SpeciesID` - numer gatunku.
4. `Ncodons` - liczba kodonów zmierzona dla danej komórki.
5. `SpeciesName` - nazwa gatunku.

Dalsze kolumny to znormalizowany ułamek kodonów poszczególnych typów w RNA danej komórki. W związku z tym, że dane są już znormalizowane, nie ma potrzeby ich skalowania.

In [ ]:
df = pd.read_csv("codon_usage.csv")
df

Mamy warning co do typu - niedobrze. Sprawdźmy typy:

In [ ]:
df.dtypes.head(20)

Kolumny `UUU` i `UUC` powinny być floatami - niedobrze. Na szczęście to tylko literówka w jednym wierszu, co można sprawdzić w danych. Usuniemy ją po prostu.

In [ ]:
df = df[pd.to_numeric(df["UUU"], errors="coerce").notnull()].copy()

df = df.copy()  # to avoid irritating SettingWithCopyWarning

df["UUU"] = df.loc[:, "UUU"].astype(float)
df["UUC"] = df.loc[:, "UUC"].astype(float)

df.dtypes.head(20)

**Zadanie 1 (1.5 punktu)**

*Preprocessing danych (wzorowany na artykule)*

1. Usuń wiersze mające mniej niż 1000 kodonów
2. Usuń wiersze dla plazmidów (wartość `plm` dla królestwa)
3. Usuń kolumny bezużyteczne dla ML: `DNAtype`, `SpeciesID`, `Ncodons`, `SpeciesName`
4. Zakoduj klasy (kolumna `Kingdom`):
  - 0 - archaea, `arc`
  - 1 - bacteria, `bct`
  - 2 - eukaryota, `pln`, `inv`, `vrt`, `mam`, `rod`, `pri`
  - 3 - bacteriophages (phages), `phg`
  - 4 - viruses (viral cells), `vrl`
5. Wyodrębnij klasy jako osobną zmienną `y`. Pamiętaj o usunięciu tej kolumny z oryginalnego DataFrame'a.

Sprawdzimy jeszcze rozkład klas:

In [ ]:
class_frequencies = y.value_counts() / y.sum()
class_frequencies = class_frequencies.sort_index()
class_frequencies.index = ["Archaea", "Bacteria", "Eukaryota", "Phages", "Viruses"]

ax = class_frequencies.plot.bar(rot=0)
plt.title("Class frequencies in %")

Jak widać, mamy nie tylko klasyfikację wieloklasową, ale na dodatek klasyfikację niezbalansowaną. W takim przypadku trzeba użyć odpowiednich metryk. Wzorując się na artykule, użyjemy **macro-averaged F1-score**. Jest to proste rozwinięcie F1-score na wiele klas.

Dla przypomnienia, w klasyfikacji binarnej, F1-score to średnia harmoniczna precyzji (precision) i czułości (recall):

$$
F1 = \frac{2}{recall^{-1} + precision^{-1}} = 2 * \frac{precision * recall}{precision + recall} = \frac{2 * TP}{2 * TP + FP + FN}
$$

Macro-averaging oznacza, że najpierw obliczamy F1-score dla każdej klasy, a później bierzemy ich średnią. Żeby taka średnia była wysoka, to musimy mieć wysoki wynik na wszystkich 5 klasach, w tym 2 mniejszościowych, więc uwzględnimy niezbalansowanie klas.

## Klasyfikator kNN

Wytrenujemy teraz podstawowy algorytm k najbliższych sąsiadów, korzystając z samych wartości domyślnych ze Scikit-learn: 5 sąsiadów, metryka euklidesowa, brak ważenia.

Zgodnie z artykułem, wykorzystamy zwykłą metodę holdout do testowania, dzieląc zbiór na treningowy i testowy w proporcjach 80%-20%. Zrobimy to ze **stratyfikacją (stratification)**, czyli tak, żeby rozkład klas był taki sam w obu podzbiorach.

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.2, random_state=0, stratify=y
)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier


clf = KNeighborsClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

y_pred_score_train = clf.predict(X_train)
y_pred_score_test = clf.predict(X_test)

f1_train = f1_score(y_train, y_pred_score_train, average="macro")
f1_test = f1_score(y_test, y_pred_score_test, average="macro")

print(f"F1 train: {100 * f1_train:.2f}%")
print(f"F1 test: {100 * f1_test:.2f}%")

Wyniki są całkiem niezłe, ale jest jeszcze miejsce na poprawę. Być może zmiana liczby sąsiadów albo metryki, albo też dodanie ważenia sąsiadów da nam jeszcze kilka procent.

W przypadku wielu klas samo agregowane F1 nie zawsze mówi jednak wszystko. Zawsze warto sprawdzić **macierz pomyłek (confusion matrix)**. Co ważne, można ją rysować albo z surową liczbą przykładów, albo znormalizować, żeby dostać wyniki w ułamku (zakres wartości 0-1). Pierwsza możliwość jest przydatna, kiedy mamy niezbalansowane klasy i patrzymy na ogólną jakość klasyfikatora. Znormalizowane wykresy są z kolei przydatne, kiedy przyglądamy się klasom z osobna.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
    clf, X_test, y_test
)
plt.title("Confusion matrix (no normalization)")
plt.show()


ConfusionMatrixDisplay.from_estimator(
    clf, X_test, y_test, normalize="true"
)
plt.title("Confusion matrix (row-normalized)")
plt.show()

Wyniki są całkiem niezłe, w końcu mamy wysokie F1. Widać jednak, że dość często mylimy klasę 0 (archeony) oraz klasę 3 (bakteriofagi) z klasą 1 (bakterie) - jest to problematyczne, ale zrozumiałe, bo tych klas jest najmniej.

Po tuningu powinniśmy otrzymać lepsze wyniki. Na dobry początek sprawdzimy różną liczbę sąsiadów. Trzeba zwrócić uwagę, że jeżeli nasza metryka ma jakieś argumenty, to trzeba użyć funkcji `make_scorer` i przekazać wynik do `GridSearchCV`.

Zwróć też uwagę, że skoro przekazujemy `n_jobs=-1` do klasyfikatora, to grid search dostaje `n_jobs=None`, żeby mieć tyle procesów, co rdzeni procesora. Przy okazji zmierzymy też czas tuningu.

In [ ]:
from time import time

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV


clf = KNeighborsClassifier(n_jobs=-1)

param_grid = {
    "n_neighbors": list(range(1, 51)),
}

multiclass_f1 = make_scorer(
    f1_score,
    average="macro", 
    greater_is_better=True,
)

cv = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring=multiclass_f1,
    cv=5,
)

time_start = time()
cv.fit(X_train, y_train)
time_end = time()

print(f"Optimal number of neighbors: {cv.best_params_['n_neighbors']}")
print(f"Tuning time: {time_end - time_start:.2f} s")

y_pred_score_train = cv.predict(X_train)
y_pred_score_test = cv.predict(X_test)

f1_train = f1_score(y_train, y_pred_score_train, average="macro")
f1_test = f1_score(y_test, y_pred_score_test, average="macro")

print(f"F1 train: {100 * f1_train:.2f}%")
print(f"F1 test: {100 * f1_test:.2f}%")

Co ciekawe, po tuningu nie tylko nie mamy lepszego wyniku, ale wręcz przeuczamy bardziej! Tak się niestety zdarza - tuning opiera się na wyniku na zbiorze walidacyjnym, które nie zawsze są dobrą estymatą. Pewnym sposobem na ominięcie tego jest zmniejszenie zakresu dla hiperparametru. Tutaj wiemy, że:
- im mniejsza liczba sąsiadów, tym większa wariancja (mocniejszy overfitting) i na odwrót
- domyślna liczba sąsiadów to 5
- przy 4 przeuczamy mocniej

Można więc po prostu sprawdzać zakres od 5 wzwyż. Do tego czeka nas jeszcze tuning metryki i ważenia sąsiadów.

**Zadanie 2 (1.5 punktu)**

*Tuning kNN*

1. Dokonaj tuningu: liczby sąsiadów (zakres [5, 25]), metryki (euklidesowa, Manhattan, cosinusowa) oraz ważenia sąsiadów (brak vs ważenie odległością). Jakie są optymalne wartości hiperparametrów?
2. Zmierz czas. Czy twoim zdaniem to długo, dla zbioru, który ma ok. 10k próbek uczących?
3. Czy macierz pomyłek dla zbioru testowego wygląda sensownie, szczególnie dla klas mniejszościowych? Czy udało się uzyskać wynik lepszy od bazowego?

// skomentuj tutaj

## Approximate Nearest Neighbors (ANN)

Postaramy się teraz przyspieszyć nasz klasyfikator, w zamian za odrobinę dokładności, za pomocą algorytmu przybliżonych najbliższych sąsiadów, a konkretnie NNDescent.

Twórcy Scikit-learn'a zdawali sobie sprawę, że algorytmy znajdowania najbliższych sąsiadów to aktywne pole rozwoju, dlatego stworzyli klasę (i interfejs) `KNeighborsTransformer`, pozwalającą integrować zewnętrzne rozwiązania. Działa ona zarówno do klasyfikacji / regresji kNN, jak i do innych algorytmów opartych o sąsiedztwo, które budują graf najbliższych sąsiadów. W przypadku jego użycia trzeba podać `metric="precomputed"`, bo obliczaniem odległości zajmuje się osobny algorytm.

Implementacja PyNNDescent korzysta właśnie z tego rozwiązania, co gwarantuje łatwość użycia - klasa `PyNNDescentTransformer` dziedziczy po `KNeighborsTransformer`. Zobaczmy, jak to działa. Wykorzystamy funkcję `make_pipeline()`, która buduje obiekt `Pipeline`, ale nie wymaga podawania nazw kolejnych elementów.

**Uwaga:** PyNNDescent dla wydajności korzysta z Numby, czyli kompilatora JIT (Just-In-Time). Dzięki temu działa to bardzo szybko, ale pierwszy import i wykonanie zajmuje sporo czasu. Wykonaj kod 2 razy - za drugim razem będzie o wiele szybciej.

In [ ]:
from time import time


def benchmark_knn_and_ann(
    sklearn_knn, 
    pynndescent_ann, 
    X_train, 
    X_test, 
    y_train, 
    y_test,
) -> None:
    # training
    start_time = time()
    sklearn_knn.fit(X_train, y_train)
    end_time = time()

    sklearn_knn_fit_time = end_time - start_time

    start_time = time()
    pynndescent_ann.fit(X_train, y_train)
    end_time = time()

    pynndescent_ann_fit_time = end_time - start_time

    # prediction

    start_time = time()
    y_pred_sklearn = sklearn_knn.predict(X_test)
    end_time = time()

    sklearn_knn_predict_time = end_time - start_time

    start_time = time()
    y_pred_pynndescent = pynndescent_ann.predict(X_test)
    end_time = time()

    pynndescent_ann_predict_time = end_time - start_time
    
    f1_knn = f1_score(y_test, y_pred_sklearn, average="macro")
    f1_ann = f1_score(y_test, y_pred_pynndescent, average="macro")

    print(f"Scikit-learn training time: {sklearn_knn_fit_time:.2f}")
    print(f"PyNNDescent training time: {pynndescent_ann_fit_time:.2f}")
    print()
    print(f"Scikit-learn prediction time: {sklearn_knn_predict_time:.2f}")
    print(f"PyNNDescent prediction time: {pynndescent_ann_predict_time:.2f}")
    print()
    print(f"Scikit-learn F1: {100 * f1_knn:.2f}%")
    print(f"PyNNDescent F1: {100 * f1_ann:.2f}%")

In [ ]:
from pynndescent import PyNNDescentTransformer
from sklearn.pipeline import make_pipeline


sklearn_knn = KNeighborsClassifier(metric="euclidean")

pynndescent_ann = make_pipeline(
    PyNNDescentTransformer(metric="euclidean", random_state=0),
    KNeighborsClassifier(metric="precomputed"),
)

benchmark_knn_and_ann(sklearn_knn, pynndescent_ann, X_train, X_test, y_train, y_test)

Czas treningu jest dłuższy - to oczekiwany wynik. Natomiast czas predykcji jest podobny - wynika to z tego, że dla metryki euklidesowej k-d tree jest bardzo wydajne. Wynik jest bardzo podobny, więc mamy dobre przybliżenie.

Sprawdźmy metrykę Manhattan, dla której typowo k-d tree radzi sobie nieco gorzej.

In [ ]:
sklearn_knn = KNeighborsClassifier(metric="manhattan")

pynndescent_ann = make_pipeline(
    PyNNDescentTransformer(metric="manhattan", random_state=0),
    KNeighborsClassifier(metric="precomputed"),
)

benchmark_knn_and_ann(sklearn_knn, pynndescent_ann, X_train, X_test, y_train, y_test)

Tutaj predykcja ANN jest już wyraźnie szybsza - co prawda dane są niezbyt duże, więc nie ma to aż takiego znaczenia, ale widać potencjał skalowalności. Dodatkowo wynik jest taki sam.

Różnice byłyby znacznie lepiej widoczne w algorytmach, które budują cały graf najbliższych sąsiadów. Poznamy je na laboratorium z redukcji wymiaru i selekcji cech.

## Zbiór danych - klasyfikacja danych binarnych

Teraz zajmiemy się klasyfikacją zbioru posiadającego same zmienne binarne, do których trzeba użyć odpowiednich metryk. Konkretnie, będzie to zbiór [HIV z benchmarku MoleculeNet](https://moleculenet.org/datasets-1), w którym na podstawie reprezentacji chemicznej molekuły trzeba przewidywać, czy jest ona inhibitorem wirusa HIV. [Inhibitory](https://en.wikipedia.org/wiki/Reaction_inhibitor) to substancje przeciwne do katalizatorów - spowalniają lub uniemożliwiają jakieś zjawisko, w naszym wypadku infekcję wirusa HIV. Przewidywanie własności molekuł to kluczowe zadanie w nowoczesnym projektowaniu leków i bardzo ważne zastosowanie ML w farmacji.

Zbiór jest już podzielony na treningowy, walidacyjny i testowy w ramach projektu [Open Graph Benchmark](https://ogb.stanford.edu/docs/graphprop/). Dane te można traktować jak klasyfikację grafów molekularnych, ale w naszym wypadku zostały już one przetworzone do postaci tabelarycznej algorytmem [ECFP4 fingerprint](https://docs.chemaxon.com/display/docs/extended-connectivity-fingerprint-ecfp.md#:~:text=Extended%2DConnectivity%20Fingerprints%20(ECFPs),a%20wide%20variety%20of%20applications.). Konkretnie, z molekuł wyciągnięto informacje o 1024 podgrafach, gdzie wartość zmiennej mówi, czy dana struktura występuje w molekule, czy też nie.

Jako że zbiór jest binarny, to nie ma potrzeby żadnego skalowania zmiennych.

W przypadku tego zbioru oficjalną metryką [w benchmarku](https://ogb.stanford.edu/docs/leader_graphprop/) jest Area Under Receiver Operating Characteristic (AUROC / ROC AUC).

**Zadanie 3 (1 punkt)**

*Ładowanie danych*

1. Wczytaj zbiory treningowy, walidacyjny i testowy. Są one w postaci plików Parquet.
2. Wyodrębnij klasę `y` do osobnych zmiennych `y_train`, `y_valid`, `y_test`.
3. Przekonwertuj macierze `X_train`, `X_valid` i `X_test` na typ bool ([przydatne pytanie na StackOverflow](https://stackoverflow.com/questions/20373039/how-do-i-convert-a-numpy-matrix-into-a-boolean-matrix)) - tak przechowywane dane zajmują mniej miejsca.
4. Narysuj wykres częstości klas w zbiorze treningowym. Jak sądzisz, czemu akurat AUROC zostało tutaj wybrane jako metryka?

**Zadanie 4 (1.5 punktu)**

*Trening klasyfikatora kNN*

1. Wytrenuj klasyfikator kNN z domyślnymi hiperparametrami, używając metryki Jaccarda. Zmierz AUROC na zbiorze testowym. Pamiętaj, żeby przekazać odpowiednie wartości do metryki (patrz [przykład w dokumentacji](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)).
2. Zmierz czas treningu oraz predykcji. Czy podczas treningu została zbudowana jakakolwiek struktura danych?
3. Dokonaj tuningu liczby sąsiadów z zakresu [1, 5, 10]. Wykorzystaj tutaj istniejący zbiór walidacyjny z pomocą `PredefinedSplit`. Może się przydać [ta odpowiedź na StackOverflow](https://stackoverflow.com/a/70155766/9472066). Do tuningu wykorzystaj metrykę AUROC (argument `scoring`).
4. Porównaj wyniki algorytmu po tuningu [z oficjalnym leaderboardem](https://ogb.stanford.edu/docs/leader_graphprop/).

// skomentuj tutaj

**Zadanie 5 (1.5 punktu)**

*Klasyfikacja z metryką Jaccarda*

1. Stwórz pipeline do klasyfikacji ANN z użyciem PyNNDescent. Wykorzystaj metrykę Jaccarda i znalezioną wcześniej optymalną liczbę sąsiadów.
2. Wytrenuj zwykłe kNN oraz ANN oraz dokonaj predykcji. Zmierz czas oraz dokładność obu metod. Czy twoim zdaniem warto dokonać w tym wypadku takiej aproksymacji?

## Wyszukiwanie z pomocą najbliższych sąsiadów

Naszym ostatnim zastosowaniem metod opartych o sąsiedztwo będzie wyszukiwanie przedmiotów. Konkretnie stworzymy wyszukiwarkę obrazów na podstawie wpisanego hasła.

Jest to zadanie **multimodalne (multimodal)**, bo operujemy na dwóch różnych modalnościach: obrazie i tekście. Takie wyszukiwarki działają następująco:
1. W fazie treningu przedmioty (obrazy) są wektoryzowane siecią neuronową i przechowywane w indeksie.
2. Użytkownik wpisuje tekst, który jest wektoryzowany za pomocą tej samej sieci neuronowej, dlatego musi być ona multimodalna.
3. Wyszukujemy najbliższych sąsiadów (obrazy-wektory) dla tekstu-wektora.

Najbardziej znanym modelem multimodalnym jest [CLIP](https://openai.com/research/clip), stworzony przez OpenAI. Jego architektura i wagi są dostępne publicznie, w przeciwieństwie do nowych rozwiązań tej firmy. Jest to model zasadniczo niewymagający fine-tuningu, działa bardzo dobrze w podstawowej formie dla wielu zadań. Został też wykorzystany jako element modelu [DALL-E 2](https://openai.com/product/dall-e-2).

Nie musisz szczegółowo wiedzieć, jak działa ten model, ale pewne podstawy są bardzo ciekawe i przydatne. Dla zainteresowanych [cały artykuł](https://cdn.openai.com/papers/Learning_Transferable_Visual_Models_From_Natural_Language.pdf) - ale ostrzegam, że jest dość zaawansowany. Wysokopoziomowo architekturę dobrze opisuje [ten artykuł](https://towardsdatascience.com/clip-the-most-influential-ai-model-from-openai-and-how-to-use-it-f8ee408958b1). CLIP składa się z dwóch enkoderów: dla tekstu i dla obrazu. Są to zwykłe pre-trenowane sieci, a konkretnie:
1. Dla obrazu - [Visual Transformer (ViT)](https://www.v7labs.com/blog/vision-transformer-guide). Jest to architektura, która używa transformerów i atencji do przetwarzania obrazów. Ideą jest, żeby pociąć obraz na sąsiadujące kawałki (patches), i traktować je jak słowa dla atencji.
2. Dla tekstu - [Transformer](https://machinelearningmastery.com/the-transformer-model/), z pewnymi modyfikacjami.

Każda z tych sieci wektoryzuje wejście, dając embedding.

![CLIP](clip.png)

Zbiorem danych była kolekcja 400 milionów par (obraz, tekst), gdzie tekst opisywał zawartość obrazu. Następnie wykonano **constrastive pre-training** - model dostaje batch $N$ par (obraz, tekst) i ma przewidzieć, które z $N \times N$ możliwych kombinacji (obraz, tekst) faktycznie wystąpiły. W tym celu maksymalizuje podobieństwo cosinusowe par, które wystąpiły w batchu, a minimalizuje podobieństwo dla $N^2 - N$ możliwych kombinacji, których nie było w batchu. W tym celu jest używana entropia krzyżowa z pewnymi modyfikacjami.

Widać więc, że model jest nienadzorowany, a zbiór danych był ogromny, więc fine-tuning nie jest nam zasadniczo potrzebny. Można go użyć do klasyfikacji, ale tym nie będziemy się zajmować. Dodatkowo odpowiednią metryką odległości dla najbliższych sąsiadów będzie podobieństwo cosinusowe.

Biblioteka [Sentence-Transformers](https://www.sbert.net/index.html) powstała dla różnych modeli transformerowych do zadań nienadzorowanych, ale dodano do niej także bardzo wygodny interfejs dla modelu CLIP - [przykład użycia](https://www.sbert.net/examples/applications/image-search/README.html). 

Jako zbiór wykorzystamy [Amazon Berkeley Objects (ABO) Dataset](https://amazon-berkeley-objects.s3.amazonaws.com/index.html) - zbiór obrazów produktów Amazona, stworzony we współpracy z University of California, Berkeley. Wykorzystamy [zminiaturyzowany zbiór](https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar), w którym obrazy mają rozmiar 256x256 pikseli, bo na nasze potrzeby jest zupełnie wystarczający.

**Zadanie 6 (3 punkty)**

*Wyszukiwarka z ANN*

1. Ściągnij [zbiór danych](https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar). Rozpakuj go obok tego notebooka. W katalogu `images/metadata` znajdziesz plik `images.csv.gz` - rozpakuj go obok tego notebooka, zawiera on ścieżki do obrazów.
2. Uzupełnij kod funkcji `vectorize_images`, która oblicza embeddingi obrazów za pomocą modelu CLIP. Przyda ci się [ten tutorial](https://www.sbert.net/examples/applications/image-search/README.html). Ustaw wartość MAX_IMAGES tak, żeby wystarczyło ci RAMu i żeby proces trwał rozsądną ilość czasu (ale co najmniej kilka-kilkanaście minut). `start_idx` oraz `end_idx` to indeksy wierszy, na których w danej chwili operujemy - po prostu przesuwamy się o `BATCH_SIZE` w pętli for.
3. Uzupełnij kod klasy `ImageSearch`:
  - w konstruktorze stwórz indeks za pomocą klasy [NNDescent](https://pynndescent.readthedocs.io/en/latest/api.html#pynndescent.pynndescent_.NNDescent), korzystając z metryki cosinusowej i `random_state=0`, może się też przydać `n_jobs=-1`
  - w metodzie `query` zwektoryzuj zapytanie tekstowe, a następ wyszukaj indeksy najbliższych sąsiadów-obrazów w indeksie (zwróć uwagę na to, co zwraca metoda `.query()` dla `NNDescent`), a następnie zwróć ścieżki do ich obrazów; zwróć uwagę na to, że `.query()` wymaga 2-wymiarowego wejścia, więc w naszym wypadku `(1, 512)`
  - w metodzie `show_images` załaduj i wyświetl obrazy; może się przydać [ten przykład](https://matplotlib.org/stable/gallery/axes_grid1/simple_axesgrid.html)
4. Przetestuj wyszukiwarkę kilkoma hasłami (metoda `.search()`). Prawdopodobnie najlepiej zadziałają hasła odpowiadające typowym produktom sprzedawanym w Amazonie.
5. Jakie widzisz zalety i wady takiego podejścia do wyszukiwania?

**Uwaga:** jeżeli masz mało RAMu lub pracujesz na Google Colab, to warto zrestartować Jupyter Notebooka, zamykając kernel. Zwolni ci to pamięć. Możesz też usunąć macierz z embeddingami (`del embeddings`) kiedy stworzysz indeks w konstruktorze `ImageSearch`.

In [ ]:
# prepare pd.Series with paths to images
image_paths = pd.read_csv("images.csv")

# "height" and "width" are original image size, let's select
# only large ones - they are probably more interesting
mask = (image_paths["height"] >= 1000) & (image_paths["width"] >= 1000)
image_paths = image_paths.loc[mask, :]

# remove columns, leaving only pd.Series with paths
image_paths = image_paths["path"]
image_paths = "images/small/" + image_paths.astype(str)

image_paths

In [ ]:
from itertools import islice

import joblib
import torch
from PIL import Image
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm


MAX_IMAGES = 10000
BATCH_SIZE = joblib.cpu_count(only_physical_cores=True)


def vectorize_images(
    image_paths: pd.Series
) -> np.ndarray:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = SentenceTransformer("clip-ViT-B-32", device=device)
    
    # batch iterator, based on Python itertools example
    def batched(iterable, n: int):
        it = iter(iterable)
        while (batch := tuple(islice(it, n))):
            yield batch
    
    # CLIP embeddings have 512 dimensions
    embeddings = np.empty((MAX_IMAGES, 512))
    
    # iterate with tqdm, it will give us a nice progress bar
    with tqdm(total=MAX_IMAGES) as pbar:
        start_idx = 0
        for batch in batched(image_paths.iloc[:MAX_IMAGES], BATCH_SIZE):
            # update end_idx; remember to stop at MAX_IMAGES!
            

            # load images with Image.open()
            
            
            # calculate embeddings
            

            # save embeddings in the embeddings array
            
            
            # update start_idx
            

            pbar.update(BATCH_SIZE)
    
    return embeddings


In [ ]:
embeddings = vectorize_images(image_paths)

In [ ]:
from pynndescent import NNDescent


class ImageSearch:
    def __init__(self, image_paths: pd.Series, embeddings: np.ndarray):
        # change to Numpy array to avoid .iloc[] and just index with []
        self.image_paths = image_paths.values
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer("clip-ViT-B-32", device=device)
        
        # create PyNNDescent index
        self.index = ...

    def search(self, text: str, n_neighbors: int = 9) -> None:
        image_paths = self.query(text, n_neighbors)
        self.show(image_paths)
        
    def query(self, text: str, n_neighbors: int = 9) -> list[str]:
        ...

    def show(self, image_paths: list[str]) -> None:
        ...


In [ ]:
image_search = ImageSearch(image_paths, embeddings)

In [ ]:
image_search.search("blue pillow")

// skomentuj tutaj